In [1]:
#Import libraries
import pandas as pd
import gmaps 
import requests

#Import API key.
from config import g_key

In [2]:
#Import CSV with city data
city_data_df = pd.read_csv("weather_data/cities.csv")
city_data_df.head()

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,0,Longyearbyen,SJ,2021-05-28 14:24:14,78.2186,15.6401,30.04,64,40,9.22
1,1,Ushuaia,AR,2021-05-28 14:24:14,-54.8000,-68.3000,31.66,93,75,20.71
2,2,Arraial Do Cabo,BR,2021-05-28 14:24:06,-22.9661,-42.0278,84.16,69,0,11.50
3,3,Hermanus,ZA,2021-05-28 14:24:15,-34.4187,19.2345,67.80,80,34,21.50
4,4,Kaitangata,NZ,2021-05-28 14:24:15,-46.2817,169.8464,43.99,92,100,3.58


In [3]:
city_data_df.dtypes

City_ID         int64
City           object
Country        object
Date           object
Lat           float64
Lng           float64
Max Temp      float64
Humidity        int64
Cloudiness      int64
Wind Speed    float64
dtype: object

In [4]:
#Config gmaps with API key
gmaps.configure(api_key = g_key)

In [24]:
#Heatmap of temp
locations = city_data_df[["Lat", "Lng"]]
max_temp = city_data_df["Max Temp"]

fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1)
heat_layer = gmaps.heatmap_layer(locations, weights = max_temp, 
                    dissipating = False, max_intensity = 250, 
                    point_radius = 4)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [25]:
#Heatmap of percent humidity
locations = city_data_df[["Lat", "Lng"]]
humidity = city_data_df["Humidity"]

fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1)
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, 
                    dissipating = False, max_intensity = 250, 
                    point_radius = 4)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [26]:
#Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
clouds = city_data_df["Cloudiness"]

fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1)
heat_layer = gmaps.heatmap_layer(locations, weights = clouds, 
                    dissipating = False, max_intensity = 250, 
                    point_radius = 4)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [69]:
#Heatmap of percent cloudiness
locations = city_data_df[["Lat", "Lng"]]
wind = city_data_df["Wind Speed"]

fig = gmaps.figure(center = (30.0, 31.0), zoom_level = 1)
heat_layer = gmaps.heatmap_layer(locations, weights = wind, 
                    dissipating = False, max_intensity = 250, 
                    point_radius = 4)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

In [70]:
#User input
min_temp = float(input("What is the minimum temperature you would like for your trip?"))
max_temp = float(input("What is the maximum temperature you would like for your trip?"))

What is the minimum temperature you would like for your trip?75
What is the maximum temperature you would like for your trip?90


In [71]:
#Filter df based on input 
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df = preferred_cities_df.dropna()
preferred_cities_df.head()        

,City_ID,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
2,2,Arraial Do Cabo,BR,2021-05-28 14:24:06,-22.9661,-42.0278,84.16,69,0,11.50
6,6,Zephyrhills,US,2021-05-28 14:24:16,28.2336,-82.1812,86.72,64,1,3.18
7,7,Kamina,CD,2021-05-28 14:20:21,-8.7386,24.9906,82.83,25,15,4.05
15,15,Le Port,RE,2021-05-28 14:24:22,-20.9373,55.2919,76.96,70,3,6.08
20,20,Belmonte,BR,2021-05-28 14:24:26,-15.8631,-38.8828,82.90,62,6,3.74


In [72]:
preferred_cities_df.isnull().sum()

City_ID       0
City          0
Country       0
Date          0
Lat           0
Lng           0
Max Temp      0
Humidity      0
Cloudiness    0
Wind Speed    0
dtype: int64

In [73]:
#Create a df to store hotel names with city, country, temp, and coor.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Lat", "Lng"]].copy()
hotel_df.tail()

,City,Country,Max Temp,Lat,Lng
539,Rio Verde De Mato Grosso,BR,84.18,-18.9181,-54.8442
540,Cockburn Town,TC,78.62,21.4612,-71.1419
542,Gorontalo,ID,77.00,0.5412,123.0595
543,Caconda,AO,76.48,-13.7333,15.0667
544,Santa Cruz,BO,78.85,-17.8000,-63.1667


In [76]:
#Set const. params for hotel search
params = {"radius": 5000, "type": "lodging", "key": g_key}

#Get hotel coord.
for index, row in hotel_df.iterrows():
        lat = row["Lat"]
        lng = row["Lng"]
        
        #Add to params
        params["location"] = f"{lat},{lng}"
        
        #Send gmaps request
        base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
        hotels = requests.get(base_url, params = params).json()
        
        #Get first result if available
        try:
            hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
            print(str(index) + " " + hotels["results"][0]["name"])
        except (IndexError): 
            print("Hotel not found ... skipping.")

2 Pousada Porto Praia
6 Roadstar Hotel Zephyrhills
7 Guest House Florvia
15 KORBEY D'OR OUEST
20 Pousada Monte Carmelo O Pão
21 Mini Hotel Central
24 VS Villa
27 Hotel do Campo
28 Flat Temporada
29 Finch Bay Galapagos Hotel
33 Cititel Penang
34 Imagination Island
35 Lindsey Hôtel
37 Salinas Park Resort Maçarico AP101
38 Jiugang Hotel - Jiayuguan
41 مقبرة ال مجمل
42 Grage Hotel Bengkulu
43 Veranda Paul et Virginie Hotel & Spa
45 Guayacan Real
50 Naivasha Hotel
62 Hotel Pigalle
66 Hotel Dona Adélia
73 AQUA Hotel Aquamarina & Spa
77 Hotel Spa Galatea
78 Heritage Park Hotel
79 Hotel Star inn Peniche
80 Edifício Bezerra
84 HS Hotsson Smart Acapulco
85 Shandrani Beachcomber Resort & Spa
90 castelo pizzaria
92 Nusa Island Retreat
94 Scoop Guest House
95 Villa Enata
104 'בית החסד - מוסדות מאורות נתן' - מיסודו של הרב נתן בוקובזה זצוק"ל
105 Halane Hotel and Restaurent
107 Salinas do Maragogi All Inclusive Resort
113 Hôtel Le Dronmi
114 Namatanai Hotel
119 Les Vagues bleues
121 Welcomhotel By ITC

In [77]:
hotel_df

,City,Country,Max Temp,Lat,Lng,Hotel Name
2,Arraial Do Cabo,BR,84.16,-22.9661,-42.0278,Pousada Porto Praia
6,Zephyrhills,US,86.72,28.2336,-82.1812,Roadstar Hotel Zephyrhills
7,Kamina,CD,82.83,-8.7386,24.9906,Guest House Florvia
15,Le Port,RE,76.96,-20.9373,55.2919,KORBEY D'OR OUEST
20,Belmonte,BR,82.90,-15.8631,-38.8828,Pousada Monte Carmelo O Pão
...,...,...,...,...,...,...
539,Rio Verde De Mato Grosso,BR,84.18,-18.9181,-54.8442,Hotel Valência
540,Cockburn Town,TC,78.62,21.4612,-71.1419,Osprey Beach Hotel
542,Gorontalo,ID,77.00,0.5412,123.0595,Maqna Hotel by Prasanthi
543,Caconda,AO,76.48,-13.7333,15.0667,NaN


In [83]:
#Add a heatmap of temp for vacation spots
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center = (30, 31), zoom_level = 1.5)
heat_layer = gmaps.heatmap_layer(locations, weights = max_temp, dissipating = False,
                                max_intensity = 300, point_radius = 4)
marker_layer = gmaps.marker_layer(locations)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))

In [84]:
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Max Temp</dt><dd>{Max Temp} °F</dd>
</dl>
"""

hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

In [85]:
#Add a heatmap of temp for vacation spots
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center = (30, 31), zoom_level = 1.5)
heat_layer = gmaps.heatmap_layer(locations, weights = max_temp, dissipating = False,
                                max_intensity = 300, point_radius = 4)
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))